## Analyzing borrowers’ risk of defaulting

The goal of the project is to prepare a report for a bank’s loan division. I need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

The evaluation of customers of based on their previous activity.

### Table of Contents <a class="anchor" id="contents"></a>

* [Importing libraries + first glance at data](#chapter1)
* [Data preprocessing](#chapter2)
    * [Missing values](#section_2_1)
    * [Data type replacement](#section_2_2)
    * [Duplicates](#section_2_3)
    * [Categorizing Data](#section_2_4)
* [Analysis](#chapter3)
* [General conclusions](#chapter4)

### Importing libraries + first glance at data <a class="anchor" id="chapter1"></a>

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
credit_scoring = pd.read_csv('credit_scoring_eng.csv')

In [4]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [6]:
credit_scoring.debt = credit_scoring.debt.astype('bool')

In [7]:
for column in credit_scoring.select_dtypes(include=['object', 'bool']):
    print(credit_scoring[column].value_counts())
    print()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
N

In [8]:
for column in credit_scoring.select_dtypes(exclude=['object', 'bool']):
    print(credit_scoring[column].describe())
    print()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

count    21525.000000
mean         0.817236
std          0.548138
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          4.000000
Name: education_id, dtype: float64

count    21525.000000
mean         0.972544
std          1.420324
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000

In the data base we have **21 525** entries. There are **12** variables including float64(2), int64(5), object(5).
- 'children' - negative figures, 20 - excessive number, need check
- 'days_employed' - missing values, negative figures; +maybe in shape of years or months it would be more convenient to look at
- 'dob_years' 101 entries has 0 year-old age
- 'education' - differently written categories, need replacement (+cross tab with 'education_id');
- 'family_status' & 'family_status_id' - seems ok
- 'gender' - strange 'XNA', need check
- 'income_type' - seems ok
- 'debt' - seems ok
- 'total_income' - not sure if I need it in the floating form, 'int' will be better; missing values
- 'purpose' - unification of purposes, now the list seems extended (e.g. 'purchase of the house' and 'purchase of the house for my family'). possible groups: car / education / buying real estate / constructing real estate / wedding

[back to contents](#contents)

### Data preprocessing <a class="anchor" id="chapter2"></a>

### Missing values <a class="anchor" id="section_2_1"></a>

In [9]:
credit_scoring['children'] = credit_scoring['children'].replace(-1,1)
credit_scoring['children'] = credit_scoring['children'].replace(20,2)

In [10]:
credit_scoring['dob_years'] = credit_scoring['dob_years'].replace(0, np.nan)

In [11]:
credit_scoring = credit_scoring[credit_scoring.gender != 'XNA']
credit_scoring['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [14]:
credit_scoring['years_employed'] = credit_scoring['days_employed'] / 365
#credit_scoring['years_employed'] = credit_scoring['years_employed'].round(decimals=1)
print('Average years employed: ',round(credit_scoring['years_employed'].mean(),2))
years_error = len(credit_scoring.loc[credit_scoring['years_employed'] > 30])
years_error_share = years_error/len(credit_scoring)*100
print('% of those who worked more than 30 years: ',round(years_error_share, 1))

Average years employed:  172.74
% of those who worked more than 30 years:  16.0


In [15]:
#replacing missing income values with median values by income types
credit_scoring['total_income'].fillna(credit_scoring.groupby(['income_type'])
                                      ['total_income'].transform('median'),inplace=True)

- 'children' - '-1', '20' replaced with '1','20'. while looking through the rows havent seen anything suspicious or extraodrinary which proves that really there could be 20 kids (also its just 0,35% of all entries - the level of possibble error is quite low). Seems to be just a typo while uploading data (*probably need to contact the team reponsible for the data supply)
- 'dob_years' - 0 convert to missing, so it wont affect the counting if I need to calculate means, for example
- So I have concerns about 'gender'. I have deleted the row with strange "XNA"-value because I couldnt retrieve the information for it in other cells and excluded from the dataframe. It is probably the error of data input.
- 'days_employed' I have changed all numbers into positive. Then I have transfered days to years to have a better understanding of what I see. Then I have checked the mean of the column because I have noticed some extreme values in the cells. The result was more than 165 years (it is impossible!). Than I have tried to check the scale of the tragedy: I have calculated the share of row with more than 30 years of work experience. 16,8% is pretty significant result, which moreover couldnt be overcome with means, medians, it is probably the problem of system parcing the data and should be reported to those responsible for data collection.
- for 'total_income' I used 'median' value of each income_type group and fill with it all missing values

### Data type replacement <a class="anchor" id="section_2_2"></a>

In [16]:
credit_scoring['total_income'] = credit_scoring['total_income'].astype('int')
credit_scoring['total_income'] = credit_scoring['total_income'].round(decimals=1)

### Duplicates <a class="anchor" id="section_2_3"></a>

In [19]:
print('# of duplicates: ',credit_scoring.duplicated().sum())

# of duplicates:  54


In [20]:
#dropping duplicates
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop = True)

In [21]:
#unification of education
credit_scoring['education_lowercase'] = credit_scoring['education'].str.lower()
credit_scoring['education'].value_counts()
education_pivot = credit_scoring.pivot_table(index=['education_lowercase'],
                                        columns = 'education_id',
                                        values='gender',
                                        aggfunc='count')
education_pivot

education_id,0,1,2,3,4
education_lowercase,,,,,
bachelor's degree,5251.0,NaN,NaN,NaN,NaN
graduate degree,NaN,NaN,NaN,NaN,6.0
primary education,NaN,NaN,NaN,282.0,NaN
secondary education,NaN,15188.0,NaN,NaN,NaN
some college,NaN,NaN,743.0,NaN,NaN


In [22]:
#Working with string variables - unifiying purposes
wordnet_lemma = WordNetLemmatizer()
 
def lemmatizing(text):
    words = nltk.word_tokenize(text)
    return [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
 
credit_scoring['lemmas'] = credit_scoring['purpose'].apply(lemmatizing)

In [23]:
list_for_counter = list() 
for lemma in credit_scoring['purpose']:
    lemma_list = lemmatizing(lemma)
    list_for_counter.extend(lemma_list)

cntr = Counter(list_for_counter)
cntr.most_common()

[('a', 5117),
 ('real', 4465),
 ('estate', 4465),
 ('car', 4308),
 ('purchase', 3306),
 ('education', 3110),
 ('to', 3071),
 ('of', 2994),
 ('transaction', 2604),
 ('property', 2539),
 ('my', 2390),
 ('buy', 2360),
 ('wedding', 2335),
 ('own', 2239),
 ('housing', 1905),
 ('house', 1904),
 ('buying', 1635),
 ('commercial', 1312),
 ('for', 1290),
 ('the', 1284),
 ('with', 1277),
 ('building', 1244),
 ('second-hand', 964),
 ('university', 948),
 ('supplementary', 907),
 ('getting', 868),
 ('ceremony', 793),
 ('having', 773),
 ('have', 769),
 ('renting', 652),
 ('out', 652),
 ('family', 638),
 ('construction', 635),
 ('renovation', 607),
 ('residential', 606),
 ('going', 496),
 ('get', 447),
 ('an', 442),
 ('profile', 436),
 ('higher', 426),
 ('become', 408),
 ('educated', 408)]

In [25]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,education_lowercase,lemmas
0,1,-8437.673028,42.0,bachelor's degree,0,married,0,F,employee,False,40620,purchase of the house,-23.116912,bachelor's degree,"[purchase, of, the, house]"
1,1,-4024.803754,36.0,secondary education,1,married,0,F,employee,False,17932,car purchase,-11.026860,secondary education,"[car, purchase]"
2,0,-5623.422610,33.0,Secondary Education,1,married,0,M,employee,False,23341,purchase of the house,-15.406637,secondary education,"[purchase, of, the, house]"
3,3,-4124.747207,32.0,secondary education,1,married,0,M,employee,False,42820,supplementary education,-11.300677,secondary education,"[supplementary, education]"
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,False,25378,to have a wedding,932.235814,secondary education,"[to, have, a, wedding]"


- 'education' - it appeared that just turning everything into lowercase helped to make a unified list witout additional editing. Also with the help of pivot table I have crossed 'education_lowercase' and 'education_id' to make sure that all the values are corresponded to each other
- 'purpose' - in order to know which groups of purposes are I made a lemmatization and counted how ofter different words appear in the column. So the groups I will end with: car, wedding, education, real estate (including commercial, housing and renovation, construction). So, I am kinda frustrated of this part because I have googled a lot and used help, and I want to ask is there an easier solution? With different kind of loop?
- I found 54 absolute duplicates and deleted them.

### Categorizing Data <a class="anchor" id="section_2_4"></a>

In [26]:
#firsly I need to decide which level of income will be a border for categories: low, medium, high
#credit_scoring.sort_values(by='total_income', ascending = False).tail(50)
#len(credit_scoring[credit_scoring['total_income'] >= 50000])
#len(credit_scoring[credit_scoring['total_income'] <= 17000])
#credit_scoring['total_income'].mean()
#credit_scoring['total_income'].median()
def income_group(total_income):
    if total_income <= 20000:
        return 'low'
    if  total_income >= 40000:
        return 'high'
    return 'medium'
#income_group(35000) - the function is working!

credit_scoring['income_group'] = credit_scoring['total_income'].apply(income_group)
credit_scoring.groupby('income_group')['total_income'].count()

income_group
high       2813
low        7764
medium    10893
Name: total_income, dtype: int64

In [27]:
mean = credit_scoring['total_income'].mean()
def income_group_check(total_income):
    if total_income <= mean:
        return 'below the mean'
    return 'above the mean'
credit_scoring['income_group_check'] = credit_scoring['total_income'].apply(income_group_check)
credit_scoring.groupby('income_group_check')['total_income'].count()

income_group_check
above the mean     8175
below the mean    13295
Name: total_income, dtype: int64

In [28]:
def purp_group(purpose):
    if 'car' in purpose:
        return 'car'
    elif 'wedding' in purpose:
        return 'wedding'
    elif 'educa' in purpose or 'university' in purpose:
        return 'education'
    elif 'hous' in purpose or 'estat' in purpose or 'propert' in purpose:
        return 'real estate'
    return 'other'

credit_scoring['purp_group'] = credit_scoring['purpose'].apply(purp_group)
credit_scoring.groupby('purp_group')['debt'].count()

purp_group
car             4308
education       4014
real estate    10813
wedding         2335
Name: debt, dtype: int64

- 'total_income' - to define the boarders for the groups by income I have checked mean/median, max/min values in the whole row. I have decided to take mean (26783) as focus. So the low income group will be lower than 20000, the high - bigger than 40000. I know it is kinda 'artifical'. If I knew how to do that, I would sort all the column and visualize it on the diagram, so I can evaluate the dispersion of values by myself, also some statistical operations could be used like precentiles or something similar.
- basing on lemmatization I made up 4 groups of purposes.

[back to contents](#contents)

### Analysis <a class="anchor" id="chapter3"></a>

> <font color='darkblue'>Is there a relation between having kids and repaying a loan on time?</font> 

In [29]:
pivot_kids = credit_scoring.pivot_table(index=['children'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_kids.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_kids['default_rate'] = pivot_kids['debts'] / pivot_kids['total']
pivot_kids.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
children,,,
4,4.0,41,0.097561
2,202.0,2128,0.094925
1,445.0,4856,0.091639
3,27.0,330,0.081818
0,1063.0,14106,0.075358
5,0.0,9,0.000000


In [30]:
def kids_group(children):
    if children == 0:
        return 'no kids'
    return 'kids'

credit_scoring['kids_group'] = credit_scoring['children'].apply(kids_group)

pivot_kids_new = credit_scoring.pivot_table(index=['kids_group'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_kids_new.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_kids_new['default_rate'] = pivot_kids_new['debts'] / pivot_kids_new['total']
pivot_kids_new.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
kids_group,,,
kids,678.0,7364,0.092070
no kids,1063.0,14106,0.075358


So what do we see here? The highest possibility of default is for families with 4 kids. But could that be seriously taken into account? I am not sure because there are just 41 cases with 4 kids. But what we definetely see here is that default_rate is significantly lower if there are no kids at all.

In general we can say that there is a correlation between having kids and repaying loan on time. But the number of kids is not that important, just having them rises the possibility of default.

> <font color='darkblue'>Is there a relation between marital status and repaying a loan on time?</font> 

In [31]:
pivot_family = credit_scoring.pivot_table(index=['family_status'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_family.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_family['default_rate'] = pivot_family['debts'] / pivot_family['total']
pivot_family.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
family_status,,,
unmarried,274.0,2810,0.097509
civil partnership,388.0,4162,0.093224
married,931.0,12344,0.075421
divorced,85.0,1195,0.071130
widow / widower,63.0,959,0.065693


In [32]:
def family_group(family_status):
    if family_status == 'married' or family_status == 'civil partnership':
        return 'not single'
    return 'single'

credit_scoring['family_group'] = credit_scoring['family_status'].apply(family_group)

pivot_family_new = credit_scoring.pivot_table(index=['family_group'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_family_new.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_family_new['default_rate'] = pivot_family_new['debts'] / pivot_family_new['total']
pivot_family_new.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
family_group,,,
single,422.0,4964,0.085012
not single,1319.0,16506,0.079910


The situation here is more diverse. So as we see highest default rate within the group of unmarried but in the same time couples in civil partnership is also close to default probability.
Additional grouping showed that in general for singles the probability of default is higher than for family-people.

But I would say that there is not strong relation between marital status and default rate. Sounds logically, that couples are better in repaying loans just because they have more opportunities. But for mature divorced people or widow/er are also good in repaying, probably because they earn more or have less problems with responsibility.

> <font color='darkblue'>Is there a relation between income level and repaying a loan on time?</font>  

In [33]:
pivot_income = credit_scoring.pivot_table(index=['income_group'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_income.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_income['default_rate'] = pivot_income['debts'] / pivot_income['total']
pivot_income.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
income_group,,,
medium,904.0,10893,0.082989
low,643.0,7764,0.082818
high,194.0,2813,0.068966


In [34]:
pivot_income_check = credit_scoring.pivot_table(index=['income_group_check'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_income_check.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_income_check['default_rate'] = pivot_income_check['debts'] / pivot_income_check['total']
pivot_income_check.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
income_group_check,,,
below the mean,1117.0,13295,0.084017
above the mean,624.0,8175,0.076330


So what we see here it is also kind of obvious result - the higher the level of income the lower the default rate. Additional comparisons I have made showed that there is significant difference between those getting more than avarege salary and those getting less. 

> <font color='darkblue'>How do different loan purposes affect on-time repayment of the loan?</font>  

In [35]:
pivot_purposes = credit_scoring.pivot_table(index=['purp_group'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_purposes.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_purposes['default_rate'] = pivot_purposes['debts'] / pivot_purposes['total']
pivot_purposes.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
purp_group,,,
car,403.0,4308,0.093547
education,370.0,4014,0.092177
wedding,186.0,2335,0.079657
real estate,782.0,10813,0.072320


In [36]:
estate_check = credit_scoring.loc[credit_scoring.purp_group == 'real estate'].copy()

def estate_group(purpose):
    if 'commerci' in purpose or 'rent' in purpose:
        return 'commercial estate'
    return 'housing'

estate_check['estate_group'] = estate_check['purpose'].apply(estate_group)
estate_check.groupby('estate_group')['debt'].count()

pivot_estate = estate_check.pivot_table(index=['estate_group'],
                                        values = 'debt',
                                        aggfunc = ['sum','count'])
pivot_estate.set_axis(['debts', 'total'], axis = 'columns', inplace=True)
pivot_estate['default_rate'] = pivot_estate['debts'] / pivot_estate['total']
pivot_estate.sort_values(by = 'default_rate', ascending = False)

,debts,total,default_rate
estate_group,,,
commercial estate,151.0,1964,0.076884
housing,631.0,8849,0.071307


We received quite interesing results: the least reliable are those who intent to buy a car and who want to study. I may assume that situation with car buyers could be explained as that car purchase leads to significant increase in costs: gas, parking, service. Along with loan, it could be hard to bear.
The second is education. Probably not all graduates succeed in finding job good enough for repaying a loan.
The most reliable clients are real estate buyers. Comparing between commercial estate and housing commercial estate buyers are less  reliable that those buying house for family. This may be explained by economical factors, companys position on the market. Business is risk.
And weddings are in the middle, but are definetely better loaners. I assume just married couples cover the loan after the wedding with the recieved money for presents.

[back to contents](#contents)

### General conclusions <a class="anchor" id="chapter4"></a>

Data check:
The data was quite messy but fine. I have deleted several rows (duplicates, error in gender column). I have replaced missing values and errors. The biggest number of missing values was for total_incomes, replaced with median by income type groups. the trickiest was 'days_employed', thanks it is not in the analysis this time but in general the revealed situation showed that there is a significant problem in data input which leads to this results. This should be checked and reported on the level of data collection.

Results:
I would say in general that there are many factors influencing default rate. The most 'dangerous' loaners are singles, with kids, car buyers with medium and low income. The most reliable are couples, no kids, property buyers.

[back to contents](#contents)